## LORA for text classification

In [ ]:
import pandas as pd, numpy as np

from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType,
    PeftModel
)

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW, 
    get_linear_schedule_with_warmup,
    DataCollatorWithPadding,
    Trainer, 
    TrainingArguments,
    AutoModelForCausalLM
)

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\n\
    percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"
    

In [ ]:
data_file = "/home/scai/phd/aiz218323/scratch/datasets/train_v2_drcat_02.csv"

full_data = pd.read_csv(data_file)
full_data = full_data[full_data['RDizzl3_seven'] == True]
full_data = full_data[["text","label"]]
full_data.reset_index(drop=True, inplace = True)

print(f"We have {len(full_data)} samples") # Number of data we have
full_data.head(5)

We have 20450 samples


,text,label
0,Cars have been around for awhile and they have...,0
1,Have you ever thought what it would be like no...,0
2,What you are about to read is going to give yo...,0
3,cars have many flaws nd and in this day and ag...,0
4,There are many advantages of limiting car usag...,0


In [ ]:
from sklearn.model_selection import train_test_split

# Split it when augmented data is ready
X_train, X_val, y_train, y_val = train_test_split(full_data["text"],
                                                  full_data["label"],
                                                  test_size=0.3,
                                                  stratify=full_data["label"],
                                                  random_state=42)
print(f"We have {len(X_train)} training samples")
print(f"We have {len(X_val)} validation samples")
print("----------------------------")
count = full_data["label"].value_counts()
print(f"Number of Essays written by Human: {count[0]}")
print(f"Number of Essays generated by LLM: {count[1]}")

X_train.reset_index(drop = True, inplace = True)
y_train.reset_index(drop = True, inplace = True)
X_val.reset_index(drop = True, inplace = True)
y_val.reset_index(drop = True, inplace = True)

We have 14315 training samples
We have 6135 validation samples
----------------------------
Number of Essays written by Human: 14250
Number of Essays generated by LLM: 6200


In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [ ]:
MODEL_PATH = 'distilbert/distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, return_dict=True, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Number of trainable parameters
print(print_number_of_trainable_model_parameters(model))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable model parameters: 66955010
all model parameters: 66955010
    percentage of trainable model parameters: 100.00%


In [ ]:
# Define the LoRA Configuration
lora_config = LoraConfig(
    r=8, # Rank Number
    lora_alpha=32, # Alpha (Scaling Factor)
    lora_dropout=0.05, # Dropout Prob for Lora
    target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return get_peft_model(model, lora_config)
    

In [ ]:
o = func()

> /tmp/ipykernel_27181/4044093400.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return get_peft_model(model, lora_config)
      4 



ipdb>  s


--Call--
> /scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg_2/lib/python3.9/site-packages/peft/mapping.py(133)get_peft_model()
    131 
    132 
--> 133 def get_peft_model(
    134     model: PreTrainedModel,
    135     peft_config: PeftConfig,



ipdb>  n


> /scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg_2/lib/python3.9/site-packages/peft/mapping.py(161)get_peft_model()
    159             the base model
    160     """
--> 161     model_config = getattr(model, "config", {"model_type": "custom"})
    162     if hasattr(model_config, "to_dict"):
    163         model_config = model_config.to_dict()



...

ipdb>  


--Return--
None
> /tmp/ipykernel_27181/492731717.py(1)<module>()
----> 1 o = func()



ipdb>  


    [... skipped 1 hidden frame]

> /home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py(3553)run_code()
   3551             finally:
   3552                 # Reset our crash handler in place
-> 3553                 sys.excepthook = old_excepthook
   3554         except SystemExit as e:
   3555             if result is not None:



ipdb>  c


In [ ]:
# Get our LoRA-enabled model
peft_model = get_peft_model(model, lora_config)

# Reduced trainble parameters
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 813314
all model parameters: 67768324
    percentage of trainable model parameters: 1.20%


In [ ]:
import datasets

In [ ]:
def tokenize_func(data):
    return tokenizer(
            data['texts'],
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )

In [ ]:
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_train,"labels":y_train}))
train_dataset = train_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)
train_dataset

Map:   0%|          | 0/14315 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 14315
})

In [ ]:
# Tokenize the Validation Data
val_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_val,"labels":y_val}))
val_dataset = val_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)

val_dataset

Map:   0%|          | 0/6135 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 6135
})

In [ ]:
def metrics(eval_prediction):
    logits, labels = eval_prediction
    pred = np.argmax(logits, axis=1)
    auc_score = roc_auc_score(labels, pred)
    return {"Val-AUC": auc_score}

train_batch_size = 32
eval_batch_size = 32

In [ ]:
# Define training Args
peft_training_args = TrainingArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/89-lora-distilbert',
    learning_rate=1e-4,
    per_device_train_batch_size=train_batch_size, # You can adjust this value base on your available GPU, You may encounter "out of memory" error if this value is too lartge
    per_device_eval_batch_size=eval_batch_size, # You can adjust this value base on your available GPU, You may encounter "out of memory" error if this value is too lartge
    num_train_epochs=5,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=10,
    weight_decay=0.01,
    seed=42,
    fp16=True, # Only use with GPU
    report_to='none'
)   

In [ ]:
import math
from sklearn.metrics import roc_auc_score

In [ ]:
# Define Optimzer
optimizer = AdamW(peft_model.parameters(), 
                  lr=1e-4,
                  no_deprecation_warning=True)

# Define Scheduler
n_epochs = peft_training_args.num_train_epochs
total_steps = n_epochs * math.ceil(len(train_dataset) / train_batch_size / 2)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps)

# Data Collator
collator = DataCollatorWithPadding(
    tokenizer=tokenizer, 
    padding="longest"
)

# Define Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset, # Training Data
    eval_dataset=val_dataset, # Evaluation Data
    tokenizer=tokenizer,
    compute_metrics=metrics,
    optimizers=(optimizer,lr_scheduler),
    data_collator=collator
)

print(f"Total Steps: {total_steps}")

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Total Steps: 1120


In [ ]:
peft_trainer.train()

Step,Training Loss,Validation Loss,Val-auc
10,0.373900,0.296158,0.872247
20,0.226500,0.210931,0.911615
30,0.171700,0.138895,0.938356
40,0.098200,0.115698,0.956599
50,0.116500,0.136511,0.955457
60,0.075100,0.095374,0.968743
70,0.063200,0.097184,0.969642
80,0.065200,0.134398,0.959780
90,0.048800,0.051861,0.980592
100,0.049000,0.111235,0.967044


/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/

TrainOutput(global_step=1120, training_loss=0.04091595844513444, metrics={'train_runtime': 3157.1177, 'train_samples_per_second': 22.671, 'train_steps_per_second': 0.355, 'total_flos': 9660184239820800.0, 'train_loss': 0.04091595844513444, 'epoch': 5.0})